## Raspagem dos resultados das provas da Maratona do Rio 2019 
Meia Maratona, Maratona, Desafio Cidade Maravilhosa, Prova de 5 e 10km e Maratoninha Gloob

In [1]:
import os
import mechanize
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from time import time
from time import sleep
from random import randint

In [90]:
url_base = 'https://globalcronometragem.com.br/evento/'
DATASET_FOLDER  = 'dataset'
MEIA_MARATONA = "meiadorio2019"
MARATONA = "maratonadorio2019"

br = mechanize.Browser()
br.addheaders = [('User-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.63 Safari/535.7')]

### Meia Maratona

In [84]:
df_meia = scrape_evento(MEIA_MARATONA, debug=True)

--> Request Masculino de 30 ate 34 anos 2.002058506011963
--> Request 0 scrape meiadorio2019 4.003050327301025
--> Request 1 scrape meiadorio2019 5.005122900009155
--> Request 2 scrape meiadorio2019 5.005055665969849
--> Request 3 scrape meiadorio2019 7.005539417266846
--> Request 4 scrape meiadorio2019 7.006804943084717
--> Request 5 scrape meiadorio2019 3.0031278133392334


In [91]:
df_maratona = scrape_evento(MARATONA, debug=True)

--> Request Masculino de 30 ate 34 anos 7.007121801376343
--> Request 0 scrape maratonadorio2019 4.004122018814087
--> Request 1 scrape maratonadorio2019 6.00475549697876
--> Request 2 scrape maratonadorio2019 1.0010428428649902
--> Request 3 scrape maratonadorio2019 6.006060838699341
--> Request 4 scrape maratonadorio2019 15.009787321090698
--> Request 5 scrape maratonadorio2019 10.01006031036377


In [92]:
df_maratona

,Nome,N. Peito,Equipe,Modalidade,Sexo,Colocação Geral,Colocação faixa etária,Tempo bruto,Tempo líquido,Categoria
0,EDMILSON DOS REIS SANTANA,7,Sem equipe,42,M,3,0,02:20:47,02:20:46,Masculino de 30 ate 34 anos
1,WILLIAM KIMBOR,19,Sem equipe,42,M,4,0,02:22:14,02:22:11,Masculino de 30 ate 34 anos
2,ANTONIO DE SOUZA DIAS,40,TJ OUTLET RUNNING,42,M,5,0,02:23:04,02:23:03,Masculino de 30 ate 34 anos
3,MEJAM REGINALD LUCIAN,26,Sem equipe,42,M,10,0,02:26:21,02:26:19,Masculino de 30 ate 34 anos
4,RENILSON VITORINO DA SILVA,3,REVEZA RACING,42,M,16,0,02:32:01,02:31:59,Masculino de 30 ate 34 anos
5,FÃBIO RAMOS DOS SANTOS,17,CAIXA / CORPORE / THEO DE SOUZA STUDIO,42,M,27,0,02:42:06,02:42:04,Masculino de 30 ate 34 anos


### Funções Utilitárias

In [3]:
# Recupera a listagem de categorias da prova do evento
def get_categorias(evento):
    
    br.open(url_base + MEIA_MARATONA)
    html = br.response().read()
    
    cat_soup = BeautifulSoup(html, "html.parser")
    select_categorias = cat_soup.find(class_ = "form-control campocategoria")
    
    if select_categorias is None:
        return None
    
    options_categorias = select_categorias.find_all("option")
    
    if options_categorias is None:
        return None
    
    options_categorias = options_categorias[1:]
    
    categorias = []
    
    for o in options_categorias:
        categorias.append(o['value'])
    
    return categorias

In [4]:
#Recupera a URL para o resultado individual de cada atleta do evento por categoria
def get_url_resultados_individuais(lista_categorias, evento, debug = False):
    
    url_resultados_individuais = []

    for categoria in lista_categorias:

        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,10))
        espera = time() - inicio

        print('--> Request ' + categoria + ' ' + str(espera))
        
        params = {'npeito': '', 'campopesquisa': '', 'categoria': categoria}

        data = urllib.parse.urlencode(params)

        br.open(url_base + evento, data = data)
        resp = br.response().read()
        soup = BeautifulSoup(resp, "html.parser")

        div_resultados = soup.find_all(class_ = "campos")

        if div_resultados is None:
            continue

        for resultado in div_resultados:
            a = resultado.find("a")
            
            if a is None:
                continue;
                
            url_individual = a['href']
            
            url_resultados_individuais.append({
                "url": url_individual,
                "categoria": categoria
            })
    
        if debug:
            break
    
    return url_resultados_individuais

In [5]:
#Recupera o resultado individual do atleta
def get_resultado_individual_detalhado(url):

    br.open(url)
    html = br.response().read()

    soup = BeautifulSoup(html, "html.parser")

    div_externa = soup.find_all(class_ = "cimaevento span12")
    
    if div_externa is None:
        return
    
    if len(div_externa) < 2:
        return
    
    div = div_externa[1]

    temp = div.text.split('\n')

    temp = list(filter(lambda x: x != '', temp))[1:-1]

    temp_dic = dict(zip(temp[::2], temp[1::2]))
    
    return temp_dic

In [88]:
#Faz a raspagem de dados do evento e retorna em um dataframe Pandas
def scrape_evento(evento, debug = False):
    
    categorias = get_categorias(evento)

    resultados_categoria = get_url_resultados_individuais(categorias, evento, debug=debug)

    data = []
    count = 0
    
    for linha in resultados_categoria:
        
        if debug == True and count > 5:
            break
        
        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,15))
        espera = time() - inicio

        print('--> Request ' + str(count) + ' scrape ' + evento + ' ' + str(espera))
        
        count += 1
        
        resultado_individual = get_resultado_individual_detalhado(linha['url'])
        resultado_individual['Categoria'] = linha['categoria']
        data.append(list(resultado_individual.values()))


    columns = ['Nome', 'N. Peito', 'Equipe', 'Modalidade', 'Sexo', 'Colocação Geral', 'Colocação faixa etária','Tempo bruto', 'Tempo líquido', 'Categoria']

    df = pd.DataFrame(data=data, columns=columns)
    
    return df